## Run evaluation on different action policies, e.g. VLA

In [ ]:
from VLABench.evaluation.evaluator import Evaluator
from VLABench.evaluation.model.policy.openvla import OpenVLA
from VLABench.evaluation.model.policy.base import RandomPolicy
from VLABench.tasks import *
from VLABench.robots import *

demo_tasks = ["select_fruit"]
unseen = True
save_dir = "~/VLABench/logs"

model_ckpt = "~/VLABench/checkpoints/openvla-7b"
# lora_ckpt = "~/VLABench/checkpoints/openvla-7b"

/home/adelli/miniconda3/envs/vlabench/lib/python3.10/site-packages/dash/_jupyter.py:30: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  _dash_comm = Comm(target_name="dash")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import os
os.environ["MUJOCO_GL"] = "egl"

In [1]:
from VLABench.tasks import *
from VLABench.utils.register import register

print(register._tasks.keys())

/home/adelli/miniconda3/envs/vlabench/lib/python3.10/site-packages/dash/_jupyter.py:30: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  _dash_comm = Comm(target_name="dash")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
odict_keys(['add_condiment', 'add_condiment_spatial', 'add_condiment_common_sense', 'add_condiment_semantic', 'insert_flower', 'insert_flower_common_sense', 'insert_flower_semantic', 'insert_flower_spatial', 'insert_bloom_flower', 'replace_wilted_flower', 'density_qa', 'friction_qa', 'magnetism_qa', 'reflection_qa', 'simple_cuestick_use', 'simple_seesaw_use', 'speed_of_sound_qa', 'thermal_expansion_qa', 'weight_qa', 'select_billiards', 'select_billiards_spatial', 'select_billiards_common_sense', 'select_billiards_semantic', 'select_book', 'select_specific_type_book', 'select_book_spatial', 'select_book_semantic', 'select_chemistry_tube', 'select_chemistry_tube_common_sense', 'select_chemistry_tube_spatial', 'select_chemistry_tube_semantic', 'select_drink', 'select_drink_common_sense', 'select_drink_spatial', 'select_dri

### Init evaluator

In [3]:
evaluator = Evaluator(
    tasks=demo_tasks,
    n_episodes=2,
    max_substeps=10,   
    save_dir=save_dir,
    visualization=False
)

Load the task episodes by seeds, instead of episodes


### Load basic random policy

In [4]:
import os
os.environ["MUJOCO_GL"] = "osmesa"
os.environ["PYOPENGL_PLATFORM"] = "osmesa"

In [ ]:
random_policy = RandomPolicy(model=None)
result = evaluator.evaluate(random_policy)

### Load policies and run evaluation

In [ ]:
# OpenVLA

policy = OpenVLA(
    model_ckpt=model_ckpt,
    # lora_ckpt=lora_ckpt,
    norm_config_file=os.path.join(os.getenv("VLABENCH_ROOT"), "configs/model/openvla_config.json"),
    device="cuda:1",
)
result = evaluator.evaluate(policy)

In [ ]:
# Pi0 (OpenPi)

from VLABench.evaluation.model.policy.openpi import OpenPiPolicy
policy = OpenPiPolicy()
result = evaluator.evaluate(policy)

## Run evaluation on different VLMs

In [ ]:
from VLABench.evaluation.model.vlm import *
from VLABench.evaluation.evaluator import VLMEvaluator

vlm_name = "GPT_4v" # valid names: ["GPT_4v", "Qwen2_VL", "InternVL2", "MiniCPM_V2_6", "GLM4v", "Llava_NeXT"]
fewshot_num = 0
task_list = ["mesh_and_texture/select_fruit"]

def initialize_model(model_name, *args, **kwargs):
    cls = globals().get(model_name)
    if cls is None:
        raise ValueError(f"Model '{model_name}' not found in the current namespace.")
    
    return cls(*args, **kwargs)

In [ ]:
vlm = initialize_model(vlm_name)
evaluator = VLMEvaluator(
    tasks=task_list,
    n_episodes=2,
    data_path=os.path.join(os.getenv("VLABENCH_ROOT"), "../dataset", "vlm"),
    save_path=os.path.join(os.getenv("VLABENCH_ROOT"), "../logs/vlm"),
)

evaluator.evaluate(vlm, few_shot_num=fewshot_num)
result=evaluator.get_final_score_dict(vlm_name)
